# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
DATA_FOLDER = 'Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [2]:
# Write your answer here

## Task 2. RNA Sequences

### Problem

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.



### Resolution method 2

By definition a microbiome is a set of microorganism in a particular environment. This term is widely used with human microbiome, witch has the goal to determine and classify all the microorganism who lives in the human body in order to determine his role in the human health and disease. In order to know the names of them, a high-throughput RNA sequencing procedure is done. This method is done several time with different environment setting, refer to the metadata file.  

The following explains the methodology that is used to resolve the problem. It is composed in 5 steps, listed below:
1. Extract & Clean all the data
2. Extract & Clean the metadata
3. Merge the data & metadata 

#### 0. Import the libraries & Define constances.

In [59]:
import pandas as pd
import numpy as np


DATA_FOLDER_MICROBIOME = 'Data/microbiome/'

#### 1. Extract & Clean all the data
In order to extract, we know that the metadata contains the name of the data files, those names will be used for extraction.

In [60]:
metadata = pd.read_excel(DATA_FOLDER_MICROBIOME+'metadata.xls')
filenames = metadata['BARCODE']
# The first file. The extraction of the first file is done outside the loop is order to instanciate the md variable.
md = pd.read_excel(DATA_FOLDER_MICROBIOME+filenames[0]+'.xls', header= None, index_col=0)

# Loop over the rest of the file names and concatenate with the previous ones.
for f in filenames[1:]:
    filename = DATA_FOLDER_MICROBIOME+f+'.xls'

    extract_file = pd.read_excel(filename, header= None, index_col=0)
    md = pd.concat([md,extract_file],axis=1,join ='outer')


Some taxons are wrapped with quote marks, we will remove them.

In [61]:
md.index = md.index.str.replace(r"\"", r"")

The first column are taxons of several microorganism. The names are the result of scientific classification, which has an hierarchical structure, see the following reference in order to know more about biological classification. (See: https://www.amentsoc.org/insects/glossary/terms/biological-classification). We will use it in order to group them together. The aim of doing so is to prepare to build the indexes. We split the taxons to group them.

The variable `n` is used to delimitate the parsing into 6 groups, one for each class. By enabling `expand`we construct the hierachical index.

In [62]:
md.index = md.index.str.split(r" ",n=5,expand = True)

In [63]:
md

1  \
Archaea  Crenarchaeota         Thermoprotei          Acidilobales             Acidilobaceae                     Acidilobus                             NaN   
                                                                              Caldisphaeraceae                  Caldisphaera                           NaN   
                                                     Desulfurococcales        Desulfurococcaceae                Ignisphaera                            7.0   
                                                                                                                Stetteria                              NaN   
                                                                                                                Sulfophobococcus                       NaN   
                                                                                                                Thermodiscus                           NaN   
                                                                                                                Thermosphaera                          NaN   
                                                                              Pyrodictiaceae                    Hyperthermus                           NaN   
                                                                                                                Pyrodictium                            NaN   
                                                                                                                Pyrolobus                              2.0   
                                                     Sulfolobales             Sulfolobaceae                     Stygiolobus                            3.0   
                                                                                                                Sulfurisphaera                         NaN   
                                                     Thermoproteales          Thermofilaceae                    Thermofilum                            3.0   
                                                                              Thermoproteaceae                  Caldivirga                             NaN   
                                                                                                                Thermocladium                          NaN   
         Euryarchaeota         Methanomicrobia       Methanocellales          Methanocellaceae                  Methanocella                           7.0   
                                                     Methanomicrobiales       Methanomicrobiaceae               Methanomicrobium                       NaN   
                                                                                                                Methanosphaerula                       NaN   
                                                                              Methanomicrobiales_incertae_sedis Methanolinea                           NaN   
                                                                              Methanospirillaceae               Methanospirillum                       NaN   
                                                     Methanosarcinales        Methanosarcinaceae                Methanimicrococcus                     1.0   
                                                                                                                Methanosalsum                          NaN   
                                                                              Methermicoccaceae                 Methermicoccus                         1.0   
                               Archaeoglobi          Archaeoglobales          Archaeoglobaceae                  Ferroglobus                            1.0   
                                                                                                                Geoglobus                              1.0   
                               Halobacteria  

We are done with the cleaning of the data. The metadata is threaded in the next section.

#### 2. Exctract & Clean the metadata
the metadata will be used to merge with the data. We already extracted the metadata file in order to extract the filename of the data (refer to section 1). Therefore, the cleaning part remains. First step is to drop the barcode because it is refering only the filenames. Unlike `GROUP` and `SAMPLE` which are the environement parameters.

In [64]:
metadata_drop = pd.DataFrame.drop(metadata,'BARCODE',axis = 1)
metadata_drop

,GROUP,SAMPLE
0,EXTRACTION CONTROL,NaN
1,NEC 1,tissue
2,Control 1,tissue
3,NEC 2,tissue
4,Control 2,tissue
5,NEC 1,stool
6,Control 1,stool
7,NEC 2,stool
8,Control 2,stool


Now, the idea is to group the environment parameters in order to create level indexes after. It start by sorting using a alphabetic order.

In [65]:
sortet_metadata = pd.DataFrame.sort_values(metadata_drop,"GROUP", axis =0)
sortet_metadata

,GROUP,SAMPLE
2,Control 1,tissue
6,Control 1,stool
4,Control 2,tissue
8,Control 2,stool
0,EXTRACTION CONTROL,NaN
1,NEC 1,tissue
5,NEC 1,stool
3,NEC 2,tissue
7,NEC 2,stool


It is sorted lexicographicaly. The `EXTRACTION CONTROL` describe the phase were the RNA sequece machine is tested. This is the reason why there is no sample taken. It is nice to have this group at first in order to remind us that is not part of the RNA experience. Next step is to push back `EXTRACTION CONTROL` in the first position.

In [66]:
group = sortet_metadata['GROUP']
sample = sortet_metadata['SAMPLE']
array = [group,sample]
indextup = list(zip(*array))
temp =indextup[4]

# Remove it.
indextup.remove(temp)

# the NAN is replace by white-space.
temp = (temp[0], ' ')
temp
# Insert in the fist position
indextup.insert(0,temp)

level_index = pd.MultiIndex.from_tuples(indextup)



#### 3.Merge the data and the metadata
The data and the metadata are ready to be merged. The metadata is the names column index. The column are not sorted to link with the metadata. First we sort the columns and then we merge.

In [67]:
md.columns= [0,1,2,3,4,5,6,7,8]
columns_reindex = [0,2,6,4,8,1,5,3,7]
md = md.reindex(columns= columns_reindex)
md

0  \
Archaea  Crenarchaeota         Thermoprotei          Acidilobales             Acidilobaceae                     Acidilobus                             NaN   
                                                                              Caldisphaeraceae                  Caldisphaera                           NaN   
                                                     Desulfurococcales        Desulfurococcaceae                Ignisphaera                            7.0   
                                                                                                                Stetteria                              NaN   
                                                                                                                Sulfophobococcus                       NaN   
                                                                                                                Thermodiscus                           NaN   
                                                                                                                Thermosphaera                          NaN   
                                                                              Pyrodictiaceae                    Hyperthermus                           NaN   
                                                                                                                Pyrodictium                            NaN   
                                                                                                                Pyrolobus                              2.0   
                                                     Sulfolobales             Sulfolobaceae                     Stygiolobus                            3.0   
                                                                                                                Sulfurisphaera                         NaN   
                                                     Thermoproteales          Thermofilaceae                    Thermofilum                            3.0   
                                                                              Thermoproteaceae                  Caldivirga                             NaN   
                                                                                                                Thermocladium                          NaN   
         Euryarchaeota         Methanomicrobia       Methanocellales          Methanocellaceae                  Methanocella                           7.0   
                                                     Methanomicrobiales       Methanomicrobiaceae               Methanomicrobium                       NaN   
                                                                                                                Methanosphaerula                       NaN   
                                                                              Methanomicrobiales_incertae_sedis Methanolinea                           NaN   
                                                                              Methanospirillaceae               Methanospirillum                       NaN   
                                                     Methanosarcinales        Methanosarcinaceae                Methanimicrococcus                     1.0   
                                                                                                                Methanosalsum                          NaN   
                                                                              Methermicoccaceae                 Methermicoccus                         1.0   
                               Archaeoglobi          Archaeoglobales          Archaeoglobaceae                  Ferroglobus                            1.0   
                                                                                                                Geoglobus                              1.0   
                               Halobacteria  

Now we merge the column index with the metadata.

In [68]:
# refer to the link above in order to know about the taxonomy.    
index_level_names  = ['Kingdom','Phylum','Class','Order','Family', 'Genes']

md.columns = level_index
md.index.names = index_level_names
md = md.fillna('Unknown')

md



EXTRACTION CONTROL  \
                                                                                                                                                                        
Kingdom  Phylum                Class                 Order                    Family                            Genes                                                   
Archaea  Crenarchaeota         Thermoprotei          Acidilobales             Acidilobaceae                     Acidilobus                                    Unknown   
                                                                              Caldisphaeraceae                  Caldisphaera                                  Unknown   
                                                     Desulfurococcales        Desulfurococcaceae                Ignisphaera                                         7   
                                                                                                                Stetteria                                     Unknown   
                                                                                                                Sulfophobococcus                              Unknown   
                                                                                                                Thermodiscus                                  Unknown   
                                                                                                                Thermosphaera                                 Unknown   
                                                                              Pyrodictiaceae                    Hyperthermus                                  Unknown   
                                                                                                                Pyrodictium                                   Unknown   
                                                                                                                Pyrolobus                                           2   
                                                     Sulfolobales             Sulfolobaceae                     Stygiolobus                                         3   
                                                                                                                Sulfurisphaera                                Unknown   
                                                     Thermoproteales          Thermofilaceae                    Thermofilum                                         3   
                                                                              Thermoproteaceae                  Caldivirga                                    Unknown   
                                                                                                                Thermocladium                                 Unknown   
         Euryarchaeota         Methanomicrobia       Methanocellales          Methanocellaceae                  Methanocella                                        7   
                                                     Methanomicrobiales       Methanomicrobiaceae               Methanomicrobium                              Unknown   
                                                                                                                Methanosphaerula                              Unknown   
                                                                              Methanomicrobiales_incertae_sedis Methanolinea                                  Unknown   
                                                                              Methanospirillaceae               Methanospirillum                              Unknown   
                                                     Methanosarcinales        Methanosarcinaceae                Methanimicrococcus                                  1   
                                                                                           

The table is ready. Lastly we check if the the indexes are unique.

In [69]:
md.index.is_unique

True

This ends the Task. 

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [12]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

NameError: name 'DATA_FOLDER' is not defined

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here